In [133]:
import numpy as np
from scipy.stats import norm, t
import math
import pandas as pd
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
def simulate_brownian_motion_paths(S0, r, sigma, T, M, I):
    """
    Simulate stock price paths using geometric Brownian motion.
    :param S0: Initial stock price.
    :param r: Risk-free rate.
    :param sigma: Volatility of the underlying asset.
    :param T: Time to maturity.
    :param M: Number of time steps .
    :param I: Number of simulation paths.
    :return: Simulated paths array.
    """
    dt = T / M
    paths = np.zeros((M + 1, I))
    paths[0] = S0
    for t in range(1, M + 1):
        Z = np.random.normal(0, np.sqrt(dt), I)
        paths[t] = paths[t - 1] * np.exp((r - 0.5 * sigma ** 2) * dt + sigma * Z)
    return paths

def simulate_student_t_paths(S0, r, sigma, T, M, I, v):
    """
    Simulate stock price paths with increments distributed according to a Student-t distribution.
    :param S0: Initial stock price.
    :param r: Risk-free rate.
    :param sigma: Volatility of the underlying asset.
    :param T: Time to maturity.
    :param M: Number of time steps .
    :param I: Number of simulation paths.
    :param v: Degrees of freedom for the Student-t distribution.
    :return: Simulated paths array.
    """
    dt = T / M
    paths = np.zeros((M + 1, I))
    paths[0] = S0
    for i in range(1, M + 1):
        Z = t.rvs(df=v, size=I)
        paths[i] = paths[i - 1] * np.exp((r - 0.5 * sigma ** 2) * dt + sigma * np.sqrt(dt) * Z)
    return paths

def calculate_delta(S, K, r, sigma, T, t):
    """
    Calculate the delta of an option using the Black-Scholes formula.
    :param S: Current stock price.
    :param K: Strike price of the option.
    :param r: Risk-free rate.
    :param sigma: Volatility of the underlying asset.
    :param T: Time to maturity.
    :param t: Current time.
    :return: Delta value.
    """
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * (T - t)) / (sigma * np.sqrt(T - t))
    return norm.cdf(d1)

def black_scholes_price(S, K, r, sigma, T, t, option_type='call'):
    """
    Calculate the Black-Scholes option price.
    :param S: Current stock price.
    :param K: Strike price.
    :param r: Risk-free interest rate.
    :param sigma: Volatility of the stock.
    :param T: Time to maturity.
    :param t: Current time.
    :param option_type: Type of the option - 'call' or 'put'.
    :return: Price of the option.
    """
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * (T - t)) / (sigma * np.sqrt(T - t))
    d2 = d1 - sigma * np.sqrt(T - t)
    if option_type == 'call':
        return S * norm.cdf(d1) - K * np.exp(-r * (T - t)) * norm.cdf(d2)
    else:  # put option
        return K * np.exp(-r * (T - t)) * norm.cdf(-d2) - S * norm.cdf(-d1)



def calculate_var_es(losses, confidence_level=0.95):
    """
    Calculate VaR and ES for a series of losses.
    :param losses: Array of losses.
    :param confidence_level: Confidence level for VaR and ES calculation.
    :return: VaR and ES values.
    """

    sorted_losses = np.sort(losses)
    var_index = int((1 - confidence_level) * len(sorted_losses))
    VaR = sorted_losses[var_index]
    ES = sorted_losses[:var_index].mean()
    return VaR, ES



def calculate_portfolio_value(price_path, K, r, sigma, T):
    """
    Calculate the delta-hedged self-financing portfolio values
    
    """
     
    M = len(price_path) - 1  # Number of time steps
    dt = T / M  # Length of each time step
    
    # Calculate initial option price, delta, and theta
    C0 = black_scholes_price(price_path[0], K, r, sigma, T, 0)
    delta_0 = calculate_delta(price_path[0], K, r, sigma, T, 0)
    theta_call = -1  # The number of call options held
    theta_stock = 0-(delta_0*theta_call)  # To ensure delta-hedging
    portfolio_values = []
    
    # Initialize portfolio value
    portfolio_value = theta_stock * price_path[0] + theta_call * C0
    portfolio_shares={'call':1,
               'stock':theta_stock}
    portfolio_values.append(portfolio_value)

    # Iterate through the price path to update the portfolio
    for t in range(1,M):
        # stock price
        
        St = price_path[t]
        
        # Calculate option price at t+1 and delta at t+1
        Ct = black_scholes_price(St, K, r, sigma, T , dt * (t))
        delta_t = calculate_delta(St, K, r, sigma, T , dt * (t))
        portfolio_value = St * portfolio_shares['stock'] + portfolio_shares['call'] * Ct
        portfolio_values.append(portfolio_value)
        # # Update theta for call and stock based on delta-hedging and self-financing condition
        # theta_call = (portfolio_shares['stock']*St+portfolio_shares['call']*Ct)/(Ct-delta_t*St) # From the delta-hedging equation
        
        # theta_stock = -theta_call * delta_t  # From the self-financing condition

        # Set up the coefficient matrix A and constant vector b
        A = np.array([[St, Ct], [1,delta_t]])
        b = np.array([St * portfolio_shares['stock'] + portfolio_shares['call'] * Ct, 0])

        # Solve for the unknowns theta_{t+1}^S and theta_{t+1}^C
        # theta_t_plus_1 = np.linalg.solve(A, b)
        try:
            portfolio_shares['stock'],portfolio_shares['call'] = np.linalg.solve(A, b)
        except:
            continue
  
 
    
    return portfolio_values

    

def calculate_portfolio_VaR(portfolioprice,confidence_level):
    """
    This function is used to calculate the portfolio loss (e.g simple return, or log return)
    """
    

    portfolioprice = pd.Series(portfolioprice)
    
    # Calculate loss (considering them as 'losses')
    loss = portfolioprice - portfolioprice.shift(-1).dropna()

    # Sort the log returns in ascending order (since losses are negative, this actually sorts them by severity)
    
    sorted_loss = loss.sort_values(ascending=True)
    # Calculate VaR given confidence level
    
    var = sorted_loss.quantile(1 - confidence_level)

    return var


def calculate_portfolio_ES(portfolioprice,confidence_level):
    """
    This function is used to calculate the expected shortfall (e.g simple return, or log return)
    """
    
    portfolioprice = pd.Series(portfolioprice)
    
    # Calculate loss (considering them as 'losses')
    loss = portfolioprice - portfolioprice.shift(-1).dropna()

    # Sort the log returns in ascending order (since losses are negative, this actually sorts them by severity)
    
    sorted_loss = loss.sort_values(ascending=True)
    # Calculate VaR given confidence level
    
    var = sorted_loss.quantile(1 - confidence_level)

    # Calculate Expected Shortfall (ES)
    es = sorted_loss[sorted_loss >= var].mean()

    return es



def quantile_removal(values):
    #remove the top 15 and bot 15 percent of the data to aviod the abnomal data generated calculation errors
    # Calculate the 15th and 85th percentiles
    values = np.array(values)
    p15 = np.percentile(values, 10)
    p85 = np.percentile(values, 90)

    # Keep only data between the 15th and 85th percentiles
    filtered_data = values[(values > p15) & (values < p85)]

    return filtered_data





In [ ]:
# Consider the different strike prices and different Vailty

results = {}

for K in tqdm([100,140,180]):

    
    for sigma in tqdm([0.2, 0.4]):

        key = f'K={K}_sigma={sigma}'
        results[key] = {
            'VaRs_Brownian': [],
            'VaRs_Student_t_1': [],
            'VaRs_Student_t_2': [],
            'VaRs_Student_t_3': [],
            'ES_Brownian': [],
            'ES_Student_t_1': [],
            'ES_Student_t_2': [],
            'ES_Student_t_3': []
        }
    
        # Parameters for simulation
        S0 = 100  # Initial stock price
        #K = 100  # Strike price
        T = 1.0  # Time to maturity (1 year)
        r = 0.05  # Risk-free rate
        #sigma = 0.2  # Volatility of the underlying asset
        M = 365  # Number of time steps 
        I = 2000  # Number of simulation paths
        v_1 = 4  # Degrees of freedom for the Student-t distribution
        v_2 = 6  # Degrees of freedom for the Student-t distribution
        v_3 = 8  # Degrees of freedom for the Student-t distribution
        dt = T / M  # Length of each time step in years

        # Simulate paths using both methods
        paths_brownian = simulate_brownian_motion_paths(S0, r, sigma, T, M, I)
        paths_student_t_1 = simulate_student_t_paths(S0, r, sigma, T, M, I, v_1)
        paths_student_t_2 = simulate_student_t_paths(S0, r, sigma, T, M, I, v_2)
        paths_student_t_3 = simulate_student_t_paths(S0, r, sigma, T, M, I, v_3)



        # VaRs_Brownian = []
        # VaRs_Student_t_1 = []
        # VaRs_Student_t_2 = []
        # VaRs_Student_t_3 = []
        # ES_Brownian=[]
        # ES_Student_t_1 = []
        # ES_Student_t_2 = []
        # ES_Student_t_3 = []

        # monte carlo simulation for all the price paths
        for i in range(I):


            # Brownian
            price_path=paths_brownian.T[i]
            portfolioprice = calculate_portfolio_value(price_path, K, r, sigma, T)
            var = calculate_portfolio_VaR(portfolioprice, confidence_level=0.95)
            ES = calculate_portfolio_ES(portfolioprice, confidence_level=0.95)
            results[key]['VaRs_Brownian'].append(var)
            results[key]['ES_Brownian'].append(ES)

            # VaRs_Brownian.append(var)
            # ES_Brownian.append(ES)

            # Student t , df = 4
            price_path=paths_student_t_1.T[i]
            portfolioprice = calculate_portfolio_value(price_path, K, r, sigma, T)
            var = calculate_portfolio_VaR(portfolioprice, confidence_level=0.95)
            ES = calculate_portfolio_ES(portfolioprice,confidence_level=0.95)
            results[key]['VaRs_Student_t_1'].append(var)
            results[key]['ES_Student_t_1'].append(ES)

            # VaRs_Student_t_1.append(var)
            # ES_Student_t_1.append(ES)

            # Student t , df = 6
            price_path=paths_student_t_2.T[i]
            portfolioprice = calculate_portfolio_value(price_path, K, r, sigma, T)
            var = calculate_portfolio_VaR(portfolioprice, confidence_level=0.95)
            ES = calculate_portfolio_ES(portfolioprice,confidence_level=0.95)
            results[key]['VaRs_Student_t_2'].append(var)
            results[key]['ES_Student_t_2'].append(ES)
            # VaRs_Student_t_2.append(var)
            # ES_Student_t_2.append(ES)


            # Student t , df = 8
            price_path=paths_student_t_3.T[i]
            portfolioprice = calculate_portfolio_value(price_path, K, r, sigma, T)
            var = calculate_portfolio_VaR(portfolioprice, confidence_level=0.95)
            ES = calculate_portfolio_ES(portfolioprice,confidence_level=0.95)
            results[key]['VaRs_Student_t_3'].append(var)
            results[key]['ES_Student_t_3'].append(ES)
            # VaRs_Student_t_3.append(var)
            # ES_Student_t_3.append(ES)

ResultTable = pd.DataFrame.from_dict({(i, j): results[i][j] 
                             for i in results.keys() 
                             for j in results[i].keys()},
                            orient='index')

In [132]:
# result data
ResultTable = pd.DataFrame.from_dict({(i, j): results[i][j] 
                             for i in results.keys() 
                             for j in results[i].keys()},
                            orient='index')
ResultTable

,0,1,2,3,4,5,6,7,8,9,...,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999
"(K=100_sigma=0.2, VaRs_Brownian)",-0.033592,-0.061254,-0.204885,-0.115617,-0.135150,-0.024037,-0.093868,-0.041426,-0.075178,-0.024620,...,-0.033339,-0.022048,-0.039765,-0.021967,-0.059596,-0.109126,-0.250468,-0.023101,-0.099942,-0.020293
"(K=100_sigma=0.2, VaRs_Student_t_1)",-0.048053,-0.097157,-0.080911,-0.020875,-0.104531,-0.022526,-0.029385,-0.169044,-0.080344,-0.045329,...,-0.175078,-0.021132,-0.138837,-0.021956,-0.020347,-0.152507,-0.070929,-0.016718,-0.109863,-0.045202
"(K=100_sigma=0.2, VaRs_Student_t_2)",-0.027025,-0.079674,-0.089333,-0.055933,-0.066058,-0.165361,-0.039547,-0.173320,-0.024367,-0.281438,...,-0.021104,-0.066665,-0.031755,-0.026873,-0.069476,-0.087919,-0.235734,-0.270694,-0.155908,-0.046850
"(K=100_sigma=0.2, VaRs_Student_t_3)",-0.030196,-0.023698,-0.025704,-0.021416,-0.232304,-0.184760,-0.022065,-0.112939,-0.025869,-0.024579,...,-0.325046,-0.237069,-0.027283,-0.235291,-0.088197,-0.049120,-0.046250,-0.184761,-0.024067,-0.052507
"(K=100_sigma=0.2, ES_Brownian)",-0.008132,-0.000954,0.012889,0.020242,-0.007034,-0.008940,-0.003994,-0.010187,-0.013586,-0.009301,...,-0.009676,-0.010740,-0.005534,-0.010203,-0.006536,0.010442,-0.003272,-0.010622,0.018649,-0.008403
"(K=100_sigma=0.2, ES_Student_t_1)",0.019383,0.177665,0.146436,0.004568,0.128298,-0.000069,0.007030,0.046088,0.024453,0.018760,...,0.096126,-0.006711,0.229709,-0.004454,-0.007819,0.084912,0.517368,-0.006551,0.104350,0.009363
"(K=100_sigma=0.2, ES_Student_t_2)",-0.008020,0.005843,0.128807,0.009245,0.014225,0.020312,-0.002410,0.050828,-0.002364,0.104717,...,-0.009260,0.031568,-0.004400,-0.006969,0.021661,0.050063,0.020232,0.041760,0.293463,0.003662
"(K=100_sigma=0.2, ES_Student_t_3)",-0.003970,-0.007559,-0.007522,-0.005916,0.013447,0.024059,-0.008158,0.032757,-0.005929,-0.008092,...,0.050182,0.103652,-0.007137,0.035520,0.013431,0.006369,-0.000919,0.021685,-0.007840,0.003470
"(K=100_sigma=0.4, VaRs_Brownian)",-0.294667,-0.047669,-0.390430,-0.307641,-0.606021,-0.546994,-0.096720,-0.090991,-0.308471,-0.095183,...,-0.242422,-0.073171,-0.058504,-0.059312,-0.292354,-0.044602,-0.059385,-0.488970,-0.118665,-0.067097
"(K=100_sigma=0.4, VaRs_Student_t_1)",-0.162930,-0.219820,-0.192277,-0.344015,-0.485498,-0.037792,-0.404794,-0.082511,-0.195672,-0.200304,...,-0.266989,-0.042978,-0.148605,-0.293637,-0.070517,-0.083828,-0.034495,-0.060610,-0.048034,-0.168558


#  plots for VaR at different strike prices, Volatility and different price paths

In [141]:
os.makedirs('plots', exist_ok=True)
for K in [100,140,180]:
    #K = 100
    for sigma in [0.2,0.4]:
    #sigma = 0.2
        key = f'K={K}_sigma={sigma}'

        # Extracting the data for the specific K and sigma
        VaRs_Brownian = results[key]['VaRs_Brownian']
        VaRs_Student_t_1 = results[key]['VaRs_Student_t_1']
        VaRs_Student_t_2 = results[key]['VaRs_Student_t_2']
        VaRs_Student_t_3 = results[key]['VaRs_Student_t_3']

        # We need to clean out some data that will generate extreme VaR which does not make sense in our model. Those extreme data is not the extreme
        # loss but the computational error from solving the equations
        VaRs_Brownian = quantile_removal(VaRs_Brownian)
        VaRs_Student_t_1 = quantile_removal(VaRs_Student_t_1)
        VaRs_Student_t_2 = quantile_removal(VaRs_Student_t_2)
        VaRs_Student_t_3 = quantile_removal(VaRs_Student_t_3)
        # Function to add text annotations to the plots
        def add_annotations(ax, mean_var, K, sigma):
            ax.text(0.05, 0.95, f'Mean VaR: {mean_var:.4f}', transform=ax.transAxes, fontsize=10, verticalalignment='top')
            ax.text(0.05, 0.85, f'Strike Price: {K}', transform=ax.transAxes, fontsize=10, verticalalignment='top')
            ax.text(0.05, 0.75, f'Sigma: {sigma}', transform=ax.transAxes, fontsize=10, verticalalignment='top')

        # Plotting the histograms
        plt.figure(figsize=(12, 8))

        ax1 = plt.subplot(2, 2, 1)
        plt.hist(VaRs_Brownian, bins=50, color='blue', alpha=0.7)
        plt.xlabel('VaR')
        plt.ylabel('Frequency')
        plt.title('Brownian Motion VaR')
        add_annotations(ax1, np.mean(VaRs_Brownian), K, sigma)

        ax2 = plt.subplot(2, 2, 2)
        plt.hist(VaRs_Student_t_1, bins=100, color='green', alpha=0.7)
        plt.xlabel('VaR')
        plt.ylabel('Frequency')
        plt.title('Student-t (v=4) VaR')
        
        add_annotations(ax2, np.mean(VaRs_Student_t_1), K, sigma)

        ax3 = plt.subplot(2, 2, 3)
        plt.hist(VaRs_Student_t_2, bins=50, color='red', alpha=0.7)
        plt.xlabel('VaR')
        plt.ylabel('Frequency')
        plt.title('Student-t (v=6) VaR')
        add_annotations(ax3, np.mean(VaRs_Student_t_2), K, sigma)

        ax4 = plt.subplot(2, 2, 4)
        plt.hist(VaRs_Student_t_3, bins=50, color='purple', alpha=0.7)
        plt.xlabel('VaR')
        plt.ylabel('Frequency')
        plt.title('Student-t (v=8) VaR')
        add_annotations(ax4, np.mean(VaRs_Student_t_3), K, sigma)

        plt.tight_layout()



        #plt.show()
        file_name = f'plots/Var_{K}_{sigma}.png'
        plt.savefig(file_name)
        plt.close()


#  plots for ES at different strike prices, Volatility and different price paths

In [146]:

# 
for K in [100,140,180]:
    #K = 100
    for sigma in [0.2,0.4]:

        key = f'K={K}_sigma={sigma}'

        # Extracting the ES data for the specific K and sigma
        ES_Brownian = results[key]['ES_Brownian']
        ES_Student_t_1 = results[key]['ES_Student_t_1']
        ES_Student_t_2 = results[key]['ES_Student_t_2']
        ES_Student_t_3 = results[key]['ES_Student_t_3']


        # We need to clean out some data that will generate extreme VaR which does not make sense in our model. Those extreme data is not the extreme
        # loss but the computational error from solving the equations
        ES_Brownian = quantile_removal(ES_Brownian)
        ES_Student_t_1 = quantile_removal(ES_Student_t_1)
        ES_Student_t_2 = quantile_removal(ES_Student_t_2)
        ES_Student_t_3 = quantile_removal(ES_Student_t_3)

        # Function to add text annotations to the plots
        def add_annotations(ax, mean_es, K, sigma):
            ax.text(0.95, 0.95, f'Mean ES: {mean_es:.4f}', transform=ax.transAxes, fontsize=10, verticalalignment='top', horizontalalignment='right')
            ax.text(0.95, 0.85, f'Strike Price: {K}', transform=ax.transAxes, fontsize=10, verticalalignment='top', horizontalalignment='right')
            ax.text(0.95, 0.75, f'Sigma: {sigma}', transform=ax.transAxes, fontsize=10, verticalalignment='top', horizontalalignment='right')

        # Plotting the histograms for Expected Shortfall
        plt.figure(figsize=(12, 8))

        # Brownian Motion ES
        ax1 = plt.subplot(2, 2, 1)
        plt.hist(ES_Brownian, bins=50, color='blue', alpha=0.7)
        plt.xlabel('Expected Shortfall')
        plt.ylabel('Frequency')
        plt.title('Brownian Motion ES')
        add_annotations(ax1, np.mean(ES_Brownian), K, sigma)

        # Student-t (v=4) ES
        ax2 = plt.subplot(2, 2, 2)
        plt.hist(ES_Student_t_1, bins=50, color='green', alpha=0.7)
        plt.xlabel('Expected Shortfall')
        plt.ylabel('Frequency')
        plt.title('Student-t (v=4) ES')
        add_annotations(ax2, np.mean(ES_Student_t_1), K, sigma)

        # Student-t (v=6) ES
        ax3 = plt.subplot(2, 2, 3)
        plt.hist(ES_Student_t_2, bins=50, color='red', alpha=0.7)
        plt.xlabel('Expected Shortfall')
        plt.ylabel('Frequency')
        plt.title('Student-t (v=6) ES')
        add_annotations(ax3, np.mean(ES_Student_t_2), K, sigma)

        # Student-t (v=8) ES
        ax4 = plt.subplot(2, 2, 4)
        plt.hist(ES_Student_t_3, bins=50, color='purple', alpha=0.7)
        plt.xlabel('Expected Shortfall')
        plt.ylabel('Frequency')
        plt.title('Student-t (v=8) ES')
        add_annotations(ax4, np.mean(ES_Student_t_3), K, sigma)

        plt.tight_layout()
        #plt.show()

        #plt.show()
        file_name = f'plots/ES_{K}_{sigma}.png'
        plt.savefig(file_name)
        plt.close()





# Mean VaR on different strike price

In [147]:
# Mean VaR on different strike price

strike_prices = [100, 140,180]
for sigma in [0.2,0.4]:
    #sigma = 0.2  # Example volatility value
    distribution_types = ['VaRs_Brownian', 'VaRs_Student_t_1', 'VaRs_Student_t_2', 'VaRs_Student_t_3']

    plt.figure(figsize=(10, 6))

    for dist in distribution_types:
        avg_vars = []
        for K in strike_prices:
            key = f'K={K}_sigma={sigma}'
            
            avg_var = np.mean(quantile_removal(results[key][dist]))
            avg_vars.append(avg_var)
        plt.plot(strike_prices, avg_vars, label=dist)

    plt.xlabel('Strike Price')
    plt.ylabel('Average VaR')
    plt.title('Average VaR for Different Strikes')
    plt.legend()
    plt.grid(True)
    #plt.show()

    #plt.show()
    file_name = f'plots/AvgVaRonStrike_{K}_{sigma}.png'
    plt.savefig(file_name)
    plt.close()


# Mean ES on different strike price

In [145]:
# Mean ES for different strike price
strike_prices = [100,  140, 180]
for sigma in [0.2,0.4]:

    plt.figure(figsize=(10, 6))

    distribution_types = ['ES_Brownian', 'ES_Student_t_1', 'ES_Student_t_2', 'ES_Student_t_3']

    for dist in distribution_types:
        avg_es = []
        for K in strike_prices:
            key = f'K={K}_sigma={sigma}'
            avg_es_value = np.mean(quantile_removal(results[key][dist]))
            avg_es.append(avg_es_value)
        plt.plot(strike_prices, avg_es, label=dist)

    plt.xlabel('Strike Price')
    plt.ylabel('Average ES')
    plt.title('Average ES for Different Strikes')
    plt.legend()
    plt.grid(True)
    #plt.show()
    
    file_name = f'plots/AvgESonStrike_{K}_{sigma}.png'
    plt.savefig(file_name)
    plt.close()
